# Grundlage des Codes
> https://pythonmana.com/2021/05/20210502184221296q.html

> **https://ofstack.com/python/45529/python-mathematical-modeling-learning-simulated-annealing-algorithm-example-analysis-of-integer-programming-problem.html**

In [2]:
import math
import random
import numpy as np
from numba import njit
from numba.typed import List
import time

# Subroutines ： Define the objective function of the optimization problem
@njit(cache=True)
def cal_Energy(X: list[int], nVar: int, mk: int) -> float: # m(k)： Punishment factor , With the number of iterations k Gradually increase
    p1 = (max(0, 6*X[0]+5*X[1]-60))**2
    p2 = (max(0, 10*X[0]+20*X[1]-150))**2
    fx = -(10*X[0]+9*X[1])
    return fx+mk*(p1+p2)

# Subroutines ： Parameter setting of simulated annealing algorithm
def ParameterSetting():
    cName = "funcOpt" # Define problem name YouCans, XUPT
    nVar = 2 # Given the number of arguments ,y=f(x1,..xn)
    xMin = [0, 0] # Given the lower bound of the search space ,x1_min,..xn_min
    xMax = [8, 8] # Given the upper limit of the search space ,x1_max,..xn_max
    tInitial = 100.0 # Set the initial annealing temperature (initial temperature)
    tFinal = 1 # Set the ending annealing temperature (stop temperature)
    alfa = 0.98 # Set the cooling parameters ,T(k)=alfa*T(k-1)
    meanMarkov = 100 # Markov Chain length , That is, the number of internal circulation runs
    scale = 0.5 # Define the search step size , It can be set to a fixed value or gradually reduced
    return cName, nVar, xMin, xMax, tInitial, tFinal, alfa, meanMarkov, scale

@njit(cache=True)
def InnerSSALoop(nVar: int,xMin: list[int],xMax: list[int],tInitial: float,tFinal: float,alfa: float,meanMarkov: int,scale: float, xInitial:np.ndarray, fxInitial:float):
    # ====== Simulated annealing algorithm initialization ======
    xNew = np.zeros((nVar))
    xNow = np.zeros((nVar))
    xBest = np.zeros((nVar))
    xNow[:] = xInitial[:] # the current solution , Set the initial solution as the current solution
    xBest[:] = xInitial[:] # the best solution , Set the current solution as the best solution
    fxNow = fxInitial # Set the objective function of the initial solution as the current value
    fxBest = fxInitial # Set the objective function of the current solution as the best value
    recordIter = [] # Number of external cycles
    recordFxNow = [] # The objective function value of the current solution
    recordFxBest = [] # The objective function value of the best solution
    recordPBad = [] # The acceptance probability of the inferior solution
    kIter = 0 # The number of iterations of the outer loop , The number of temperature states
    totalMar = 0 # A total of Markov Chain length
    totalImprove = 0 # The number of improvements for fxBest
    nMarkov = meanMarkov # Fixed length Markov chain
    # ====== Start simulated annealing optimization ======
    # Outer loop , Until the current temperature reaches the end temperature
    tNow = tInitial # Initialize the current temperature (current temperature)
    while tNow >= tFinal: # Outer loop , Until the current temperature reaches the end temperature
        # At the current temperature , Do it enough times (nMarkov) To achieve thermal equilibrium
        kBetter = 0 # The number of times a good solution is obtained
        kBadAccept = 0 # The number of times a bad solution is accepted
        kBadRefuse = 0 # The number of times a bad solution is rejected
        # --- Inner loop , The number of cycles is Markov Chain length
        for k in range(nMarkov): # Inner loop , The number of cycles is Markov Chain length
            totalMar += 1 # total Markov Chain length counter
            # --- New solutions
            # New solutions ： A new solution is generated by random perturbation near the current solution , The new solution must be in [min,max] Within the scope of
            # programme 1： Only right n One of the variables is perturbed , Other n-1 Two variables remain unchanged
            xNew[:] = xNow[:]
            v = random.randint(0, nVar-1) # produce [0,nVar-1] Random number between
            xNew[v] = round(xNow[v] + scale * (xMax[v]-xMin[v]) * random.normalvariate(0, 1))
            # Satisfy that the decision variable is an integer , Use the simplest solution ： The resulting new solution is rounded to the nearest whole
            xNew[v] = max(min(xNew[v], xMax[v]), xMin[v]) # Make sure the new solution is [min,max] Within the scope of
            # --- Calculate the objective function and the energy difference
            # Call subfunction cal_Energy Calculate the objective function value of the new solution
            fxNew = cal_Energy(xNew, nVar, kIter)
            deltaE = fxNew - fxNow
            # --- Press Metropolis The criteria accept new interpretations
            # Accept judgment ： according to Metropolis The criteria decide whether to accept the new interpretation
            if fxNew < fxNow: # Better solution ： If the objective function of the new solution is better than the current solution , Then accept the new explanation
                accept = True
                kBetter += 1
            else: # Tolerance solution ： If the objective function of the new solution is worse than the current solution , The new solution is accepted with a certain probability
                pAccept = math.exp(-deltaE / tNow) # Calculate the state transition probability of the tolerant solution
                if pAccept > random.random():
                    accept = True # Accept the bad solution
                    kBadAccept += 1
                else:
                    accept = False # Refuse inferior solutions
                    kBadRefuse += 1
            # Save the new solution
            if accept == True: # If you accept the new explanation , The new solution is saved as the current solution
                xNow[:] = xNew[:]
                fxNow = fxNew
                if fxNew < fxBest: # If the objective function of the new solution is better than the optimal solution , Then the new solution is saved as the optimal solution
                    fxBest = fxNew
                    xBest[:] = xNew[:]
                    totalImprove += 1
                    scale = scale*0.99 # Variable search step size , Gradually reduce the search scope , Improve search accuracy
        # --- Data processing after the end of the inner loop
        # Complete the current temperature search , Save data and output
        pBadAccept = kBadAccept / (kBadAccept + kBadRefuse) # The acceptance probability of the inferior solution
        recordIter.append(kIter) # The current number of external loops
        recordFxNow.append(round(fxNow, 4)) # The objective function value of the current solution
        recordFxBest.append(round(fxBest, 4)) # The objective function value of the best solution
        recordPBad.append(round(pBadAccept, 4)) # The objective function value of the best solution
        #if kIter%10 == 0: # Modular arithmetic , The remainder of the quotient
          #  print('i:{},t(i):{:.2f}, badAccept:{:.6f}, f(x)_best:{:.6f}'.\
           #       format(kIter, tNow, pBadAccept, fxBest))
        # Slow down to a new temperature , The cooling curve ：T(k)=alfa*T(k-1)
        tNow = tNow * alfa
        kIter = kIter + 1
        fxBest = cal_Energy(xBest, nVar, kIter) # Because the penalty factor increases after iteration , Then we need to reconstruct the augmented objective function
    # ====== End the simulated annealing process ======
    return totalImprove,kIter,xBest,fxBest,fxNow,recordIter,recordFxNow,recordFxBest,recordPBad


# Simulated annealing algorithm
def OptimizationSSA(nVar: int,xMin: list[int],xMax: list[int],tInitial: float,tFinal: float,alfa: float,meanMarkov: int,scale: float):
    # ====== Initialize the random number generator ======
    randseed = random.randint(1, 100)
    random.seed(time.time()) # Random number generator set seed , It can also be set to a specified integer
    # ====== The initial solution of the optimization problem is generated randomly ======
    xInitial = np.zeros((nVar)) # initialization , Create array
    for v in range(nVar):
        # xInitial[v] = random.uniform(xMin[v], xMax[v]) # produce [xMin, xMax] Random real numbers of ranges
        xInitial[v] = random.randint(xMin[v], xMax[v]) # produce [xMin, xMax] The random integer of the range
    # Call subfunction cal_Energy Calculate the objective function value of the current solution
    fxInitial = cal_Energy(xInitial, nVar, 1) # m(k)： Punishment factor , The initial value is 1
    print('x_Initial:{:.0f},{:.0f},\tf(x_Initial):{:.6f}'.format(xInitial[0], xInitial[1], fxInitial))
    
    [totalImprove,kIter,xBest,fxBest,fxNow,recordIter,recordFxNow,recordFxBest,recordPBad]\
    = InnerSSALoop(nVar,xMin,xMax,tInitial, tFinal, alfa, meanMarkov, scale, xInitial, fxInitial)

    print('improve:' + str(totalImprove))
    return kIter,xBest,fxBest,fxNow,recordIter,recordFxNow,recordFxBest,recordPBad

# Results check and output
def ResultOutput(cName,nVar,xBest,fxBest,kIter,recordFxNow,recordFxBest,recordPBad,recordIter):
    # ====== Check and output the optimization results ======
    fxCheck = cal_Energy(xBest, nVar, kIter)
    if abs(fxBest - fxCheck)>1e-3: # Test the objective function
        print("Error 2: Wrong total millage!")
        return
    else:
        print("\nOptimization by simulated annealing algorithm:")
        for i in range(nVar):
            print('\tx[{}] = {:.1f}'.format(i,xBest[i]))
        print('\n\tf(x) = {:.1f}'.format(cal_Energy(xBest,nVar,0)))
    return

# The main program
def main():
    # Parameter setting , Parameter definition of optimization problem , Parameter setting of simulated annealing algorithm
    [cName, nVar, xMin, xMax, tInitial, tFinal, alfa, meanMarkov, scale] = ParameterSetting()
    # print([nVar, xMin, xMax, tInitial, tFinal, alfa, meanMarkov, scale])
    # Simulated annealing algorithm
    [kIter,xBest,fxBest,fxNow,recordIter,recordFxNow,recordFxBest,recordPBad] \
    = OptimizationSSA(nVar,xMin,xMax,tInitial,tFinal,alfa,meanMarkov,scale)
    # print(kIter, fxNow, fxBest, pBadAccept)
    # Results check and output
    ResultOutput(cName, nVar,xBest,fxBest,kIter,recordFxNow,recordFxBest,recordPBad,recordIter)

if __name__ == '__main__':
    main()


x_Initial:8,8,	f(x_Initial):8732.000000
improve:3

Optimization by simulated annealing algorithm:
	x[0] = 8.0
	x[1] = 2.0

	f(x) = -98.0


In [3]:
# function checks if order must not be changed
@njit(cache=True)
def mustPrecede(cust_i:int,dish_i:int,cust_j:int,dish_j:int):
    if cust_i == cust_j:
        return dish_i < dish_j
    elif cust_i < cust_j:
        return dish_i <= dish_j
    else:
        return dish_j < dish_i and dish_i <= dish_j + (cust_j - cust_i -1)

In [48]:
# function to move all customers for a given order to process
@njit(cache=True)
def serveCustomer(n:int, pos_waiter:float, custToServ:int, positions:list[float], orders: list[list[float]], t_now:float, v_waiter:float, v_cust:float, t_serving:float):
    newPositions = [p for p in positions]
    newOrders = orders
    #newOrders = [[o for o in ol] for ol in orders]

    servedOrderAt = newOrders[custToServ][0]
    delta_t= max(servedOrderAt-newPositions[custToServ]/v_cust, abs(servedOrderAt-pos_waiter)/v_waiter) + t_serving # time required to serve customer
    t_now += delta_t
    pos_waiter = servedOrderAt
    # do not remove order yet as it is used to determine blocking conditions

    # update customer position of each of the n customers
    if len(newOrders[0]) > 0: # check if orders are still in cue
        newPositions[0] = min(newOrders[0][0], newPositions[0] + delta_t * v_cust)
    else:
        newPositions[0] = newPositions[0] + delta_t * v_cust # no further orders

    for i in range(1,n):
        if len(newOrders[i]) > 0: # check if orders are still in cue
            newPositions[i] = min(newOrders[i][0], newPositions[i] + delta_t * v_cust, newPositions[i-1] - 1)
        else:
            newPositions[i] = min(newPositions[i] + delta_t * v_cust, newPositions[i-1] - 1) # no further orders

    servedOrderAt = newOrders[custToServ][0]
    newOrders[custToServ] = newOrders[custToServ][1:]

    return servedOrderAt, pos_waiter, newOrders, newPositions, t_now

In [49]:
@njit(cache=True)
def getCandidates(n:int, orders:list[list[float]]):
    candidates=[]
    if len(orders[0]) > 0: # first customer is always possible if order are present
        candidates.append(0)

    for j in range(1,n):
        if len(orders[j]) > 0:
            posCandidate = True
            for i in range(0,j):
                if len(orders[i]) > 0 and mustPrecede(i,orders[i][0],j,orders[j][0]):
                    posCandidate=False
                    break
            if posCandidate:
                candidates.append(j)
    return candidates

In [50]:
class State():
    def __init__(self):
        self.served = []
        self.pos_waiter = 0
        self.positions = []
        self.orders = [[]]
        self.t_now = 0


In [52]:
# Beamsearch
def beamSearch(beta:int, n:int, pos_waiter:float, positions:list[float], orders:list[list[float]], t_now:float, v_waiter:float, v_cust:float, t_serving:float) -> list[int]:
    noOfOrders = sum([len(e) for e in orders])

    curLevel = []#  list containing states at current level
    
    # initial state of tree
    s = State()
    s.pos_waiter = pos_waiter
    s.positions = positions
    s.orders = orders
    s.t_now = t_now
    curLevel.append(s)

    for _ in range(noOfOrders):
        newLevel = [] # list containing states at new level
        for state in curLevel:
            candidates=getCandidates(n,state.orders)
            for custToServ in candidates:
                newState = State()
                [servedOrderAt, newState.pos_waiter, newState.orders, newState.positions, newState.t_now] \
                    = serveCustomer(n, state.pos_waiter, custToServ, state.positions, state.orders, state.t_now, v_waiter, v_cust, t_serving)
                newState.served = state.served + [[custToServ, servedOrderAt]]
                newLevel.append(newState)
            
        # reduce tree size if necessary
        if len(newLevel) > beta:
            threshold = sorted([state.t_now for state in newLevel])[10]
            reducedLevel = filter(lambda state: state.t_now >= threshold, newLevel)
            curLevel = reducedLevel
        else:
            curLevel = newLevel
    
    return curLevel

In [54]:
# Test of Beams Search
n=3
m=4
orders=[[1,3], [1], [2]]
nbList = List()
[nbList.append(o) for o in orders]
positions = list(range(-1,-4,-1))
pos_waiter = 0
servedOrder=[0,1,0,2]
v_waiter = 1
v_cust = 1
t_serving = 2
t_now = 0
beta = 10
a = beamSearch(beta, n, pos_waiter, positions, nbList, t_now, v_waiter, v_cust, t_serving)
print(a)

/home/ruediger/anaconda3/lib/python3.9/site-packages/numba/core/ir_utils.py:2152: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'orders' of function 'getCandidates'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../../../tmp/ipykernel_70689/3867542271.py", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


TypeError: Failed in nopython mode pipeline (step: native lowering)
cannot reflect element of reflected container: reflected list(reflected list(int64)<iv=None>)<iv=None>


In [124]:
## generate customers
import random
import time

m = 10 #number of counters
n = 10 #number of customers

gamma = [1, 10] # interval for dishes per customer 

orders=[] #list of all orders
#random.seed(42)
for _ in range(n):
    orders.append(sorted(random.sample(range(m), random.randint(gamma[0], gamma[1]))))
    #temp=[]
    #for _ in range(random.randint(gamma[0], gamma[1])):
    #    temp.append(random.randint(0,9))
    #orders.append(sorted(temp))

positions = list(range(-n,0)) # list of customer positions
pos_server = 0 # position of waiter

orders

[[0, 4, 7],
 [2],
 [4, 7],
 [0, 1, 2, 3, 4, 5, 6, 7],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 4, 5, 7, 8],
 [2, 3, 6],
 [0, 1, 6],
 [1, 2, 4, 5, 6, 7, 8],
 [0]]

In [123]:
# Test of serve Customer function
n=3
m=4
orders=[[1,3], [1], [2]]
nbList = List()
[nbList.append(o) for o in orders]
positions = list(range(-1,-4,-1))
pos_waiter = 0
servedOrder=[0,1,0,2]
v_waiter = 1
v_cust = 1
t_serving = 2
t_now = 0

for custToServ in servedOrder:
    [servedOrderAt, pos_waiter, orders, positions, t_now] = serveCustomer(n, pos_waiter, custToServ, positions, nbList, t_now, v_waiter, v_cust, t_serving)
    print(t_now)

4.0
7.0
11.0
14.0
